In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [34]:
import pandas as pd

link = 'https://drive.google.com/open?id=1xddP7zLBz9qi9nRYv9NgnRFc0iBCJDJP' # The shareable link

fluff, id = link.split('=')

print (id) # Verify that you have everything after '='

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Train.csv')  
df3 = pd.read_csv('Train.csv')

# Dataset is now stored in a Pandas Dataframe

1xddP7zLBz9qi9nRYv9NgnRFc0iBCJDJP


In [35]:
link = 'https://drive.google.com/open?id=18ee07jlMp08yI0U9OJ1PWkVtanZ-foyf' # The shareable link

fluff, id = link.split('=')

print (id) # Verify that you have everything after '='

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('Test.csv')  
df_test = pd.read_csv('Test.csv')

# Dataset is now stored in a Pandas Dataframe

18ee07jlMp08yI0U9OJ1PWkVtanZ-foyf


In [0]:
import pandas as pd
import numpy as np
from scipy import interpolate
def Interpolation(df,features):
    for i in range(df.shape[0]):
        arr = df.iloc[i].values
        img1 = arr.reshape((32,32))
        x = np.arange(0, img1.shape[1])
        y = np.arange(0, img1.shape[0])
        #mask invalid values
        img1 = np.ma.masked_invalid(img1)
        xx, yy = np.meshgrid(x, y)
        #get only the valid values
        x1 = xx[~img1.mask]
        y1 = yy[~img1.mask]
        newarr = img1[~img1.mask]
        GD1 = interpolate.griddata((x1, y1), newarr.ravel(), (xx, yy), method='cubic', fill_value= np.nanmean(arr))
        GD1 = GD1.reshape((1,features))
        df.loc[i:i,:] = GD1
    return df

In [19]:
import numpy as np
import pandas as pd
from scipy import interpolate


X = df3.iloc[:, -1025:-1].values  
y = df3.iloc[:, -1].values 
df4= pd.DataFrame(data=X)
gd=Interpolation(df4,1024)
print (gd)

             0           1           2           3           4           5     \
0      130.006910  128.450292  211.000000  222.000000  221.267661  210.886868   
1      138.467961  140.128655  161.000000  162.000000  193.000000  209.901434   
2      110.415480  115.216374  115.216374  246.000000  247.000000  247.000000   
3      231.000000  228.000000  225.206967  222.190551  223.000000  227.651394   
4      157.575642  230.000000  230.000000  231.000000  231.508208  232.000000   
5       36.000000   29.000000   25.289315   22.888802   25.000000   32.091676   
6      108.268376  153.000000  153.000000  155.000000  155.778576  155.592582   
7      119.110755   32.000000   34.000000   45.872085   61.000000   74.000000   
8       92.257709   51.000000   50.000000   66.000000   59.968444   44.000000   
9      254.000000  153.611328  153.611328  153.611328  250.000000  250.000000   
10     149.288458  153.021442  240.000000  239.000000  239.048016  239.000000   
11      74.128047   13.00000

In [49]:
X_test = df_test.iloc[:, -1024:].values  
#y_test = df_test.iloc[:, -1].values 
df_test2= pd.DataFrame(data=X_test)
gd_test=Interpolation(df_test2,1024)
print (gd_test)

            0           1           2           3           4           5     \
0      32.726009   55.811641   59.000000   41.000000   30.000000   51.000000   
1     235.000000  232.000000  212.525908  221.358152  233.000000  233.000000   
2      93.971726  150.000000  159.886531  157.833284  154.000000  154.000000   
3      97.000000  106.216462   96.000000   97.000000   96.000000   93.794606   
4     106.000000  136.904149  134.232760  123.000000  119.000000  127.780202   
5      46.000000   76.551277   93.000000   80.565780   65.284818   48.692550   
6      13.000000   19.729008   28.767955   38.000000   46.449988   53.242388   
7      46.000000   50.000000   52.181282   56.403972   60.924676   64.000000   
8     125.000000  167.165461  165.113380  142.754609  124.000000  119.813259   
9      23.787196   44.758475   62.637795   77.145016   88.000000  101.000000   
10     15.561470   22.000000   21.605959   28.000000   32.183051   29.645549   
11     -4.698010   15.050665   42.025108

In [0]:
#from future import print_function
import keras
from keras.datasets import mnist
from keras.layers import Dense, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers
from keras.callbacks import LearningRateScheduler
from sklearn.model_selection import train_test_split
import matplotlib.pylab as plt
import pandas as pd
import csv as csv
import numpy as np

In [68]:
batch_size = 128
num_classes = 10
epochs = 10

# input image dimensions
img_x, img_y = 32, 32

# load the MNIST data set, which already splits into train and test sets for us
x_train = gd.iloc[:,:].values 
x_test = gd_test.iloc[:,:].values 

# reshape the data into a 4D tensor - (sample_number, x_img_size, y_img_size, num_channels)
# because the MNIST is greyscale, we only have a single channel - RGB colour images would have 3
x_train = x_train.reshape(x_train.shape[0], img_x, img_y, 1)
x_test = x_test.reshape(x_test.shape[0], img_x, img_y, 1)
input_shape = (img_x, img_y, 1)

# convert the data to the right type
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices - this is for use in the
# categorical_crossentropy loss below
y_train = df3.iloc[:, -1].values 
#y_test = df3.iloc[30000:, -1].values 

y_train = keras.utils.to_categorical(y_train, num_classes)
#y_test = keras.utils.to_categorical(y_test, num_classes)

x_train shape: (48500, 32, 32, 1)
48500 train samples
9700 test samples


In [69]:
weight_decay = 1e-4
model = Sequential()
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
 
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.3))
 
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.4))
 
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))
 
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 32, 32, 32)        320       
_________________________________________________________________
activation_19 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_19 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
activation_20 (Activation)   (None, 32, 32, 32)        0         
_________________________________________________________________
batch_normalization_20 (Batc (None, 32, 32, 32)        128       
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 16, 16, 32)        0         
__________

In [0]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adam(),
              metrics=['accuracy'])

In [71]:
class AccuracyHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.acc = []

    def on_epoch_end(self, batch, logs={}):
        self.acc.append(logs.get('acc'))

history = AccuracyHistory()

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          
          callbacks=[history])
#score = model.evaluate(x_test, y_test, verbose=0)
#print('Test loss:', score[0])
#print('Test accuracy:', score[1])
#plt.plot(range(1, 11), history.acc)
#plt.xlabel('Epochs')
#plt.ylabel('Accuracy')
#plt.show()

Epoch 1/10
48500/48500 [==============================] - 11s 218us/step - loss: 2.1451 - acc: 0.3602
Epoch 2/10
48500/48500 [==============================] - 9s 179us/step - loss: 1.5216 - acc: 0.5116
Epoch 3/10
48500/48500 [==============================] - 9s 179us/step - loss: 1.3430 - acc: 0.5754
Epoch 4/10
48500/48500 [==============================] - 9s 179us/step - loss: 1.2566 - acc: 0.6081
Epoch 5/10
48500/48500 [==============================] - 9s 179us/step - loss: 1.1921 - acc: 0.6318
Epoch 6/10
48500/48500 [==============================] - 9s 179us/step - loss: 1.1392 - acc: 0.6534
Epoch 7/10
48500/48500 [==============================] - 9s 178us/step - loss: 1.1042 - acc: 0.6695
Epoch 8/10
48500/48500 [==============================] - 9s 178us/step - loss: 1.0613 - acc: 0.6852
Epoch 9/10
48500/48500 [==============================] - 9s 182us/step - loss: 1.0320 - acc: 0.6971
Epoch 10/10
48500/48500 [==============================] - 9s 182us/step - loss: 1.0048 - 

In [72]:
import numpy as np
predictions = model.predict(x_test)
pre_arr=np.arange(9700)
for i in range(9700):
  pre_arr[i]=np.argmax(predictions[i])
print (pre_arr)
np.savetxt("predictions.csv", pre_arr, delimiter=",")

[6 0 1 ... 8 7 4]
